# Import

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [3]:
train_df = pd.read_csv("../../data/train.csv")
test_df = pd.read_csv("../../data/test.csv")  
submission = pd.read_csv("../../data/sample_submission.csv")  

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      54609 non-null  object
 1   분류      54609 non-null  object
 2   제목      54609 non-null  object
 3   키워드     54609 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   제목      23405 non-null  object
 2   키워드     23405 non-null  object
dtypes: object(3)
memory usage: 548.7+ KB


In [6]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23405 entries, 0 to 23404
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      23405 non-null  object
 1   분류      23405 non-null  object
dtypes: object(2)
memory usage: 365.8+ KB


In [7]:
len(train_df['분류'].value_counts())

56

타겟변수 -> '분류'이고 종류 56가지

In [8]:
test_df.head()

,ID,제목,키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대..."
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바..."
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의..."
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경..."


In [9]:
train_df['키워드'][0]

'용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화재단,용인문화재단,제공,시민들,심신,여유,안정,인문학,콘서트,개최,재단,18일,큰어울마당,용인시,평생,학습관,인문학,콘서트,뮤지컬,이야기,인문학,콘서트,방송,매체,왕성,활동,뮤지컬,평론가,교수,원종원,순천향대,진행,이건명,김소향,김보경,유건우,배우들,정상급,뮤지컬,배우,라이브,무대,동시,만끽,공연,공연,홍지민,뮤지컬,배우,참여,라이브,무대,선사,영화,뮤지컬,합성어,무비컬,주제,맘마미아,드림걸즈,유명,작품,교수,위트,해설,예정,공연,초등학생,관람,초등학생,가능,티켓,가격,전석,공연,용인문화재단,누리집,확인'

## 전처리

In [10]:
import re
from collections import Counter

# 영어+한글, 숫자+한글, 숫자+영어, 한글+영어, 한글+숫자, 영어+숫자, 한자, 숫자+퍼센트, 숫자, 영어 단어들을 제거하는 함수
removed_keywords = []

def remove_invalid_keywords(keywords):
    pattern = re.compile(r'[A-Za-z]+[가-힣\u4E00-\u9FFF]+|[0-9]+[가-힣\u4E00-\u9FFF]+|[0-9]+[A-Za-z]+|[가-힣]+[A-Za-z\u4E00-\u9FFF]+|[가-힣]+[0-9]+|[A-Za-z]+[0-9]+|[\u4E00-\u9FFF]+|[0-9]+(\.[0-9]+)?%|[0-9]+|[A-Za-z]+')
    valid_keywords = []
    for word in keywords.split(','):
        word = word.strip()
        if pattern.match(word):
            removed_keywords.append(word)
        else:
            valid_keywords.append(word)
    return ', '.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드'] = train_df['키워드'].apply(remove_invalid_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_invalid_keywords)

# 결과 출력
print("\n수정된 데이터프레임:")
print(train_df)

# 제거된 단어들 출력
print("\n제거된 단어들:")
print(removed_keywords[:10])


수정된 데이터프레임:
                ID        분류                                     제목  \
0      TRAIN_00000  문화:전시_공연  용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최   
1      TRAIN_00001        지역           용인 농촌테마파크, 7~8월 단체체험객 체험료 지원   
2      TRAIN_00002        지역        용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장   
3      TRAIN_00003        지역        수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대   
4      TRAIN_00004        국제      용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할   
...            ...       ...                                    ...   
54604  TRAIN_54604        국제  용인 아파트서 30대 여성, 아들 딸 함께 추락 "극단 선택 추정"   
54605  TRAIN_54605  사회:교육_시험                    용인시, '위탁부모 보수교육' 실시   
54606  TRAIN_54606        지역          용인시, '플랫폼 시티' 국토부에 사업인정 협의 신청   
54607  TRAIN_54607        지역      용인시 이동읍 주민자치위원회, 저소득 20가구에 밑반찬 지원   
54608  TRAIN_54608        지역                 용인시-용인시공무원노조, 국무총리상 수상   

                                                     키워드  
0      용인문화재단, 인문학, 콘서트, 뮤지컬, 이야기, 개최, 인문학, 콘서트, 뮤지컬,...  
1      용인, 농촌, 테

In [11]:
# 제거된 단어들의 빈도 계산 및 상위 20개 출력
keyword_counter = Counter(removed_keywords)
top_10_keywords = keyword_counter.most_common(20)

print("\n제거된 단어들 상위 20개:")
for keyword, count in top_10_keywords:
    print(f"{keyword}: {count}")


제거된 단어들 상위 20개:
코로나19: 20593
A씨: 13186
Yongin: 11410
City: 7548
1만: 6452
city: 5862
SK하이닉스: 5605
AI: 5271
Google: 5108
1년: 4816
SK: 4813
GTX: 4286
LH: 4246
Translate: 4229
Lee: 4053
3년: 4034
2년: 3950
Gyeonggi: 3859
B씨: 3770
6월: 3518


## 대분류 기준으로 공통단어 제거

In [44]:
# '키워드' 열의 모든 단어들을 리스트로 모으기
all_keywords = []
train_df['키워드'].apply(lambda x: all_keywords.extend(x.split(',')))

# 모든 단어들의 빈도 계산
keyword_counts = pd.Series(all_keywords).value_counts()

# 갯수가 1개 이하인 키워드들 필터링
low_frequency_keywords = keyword_counts[keyword_counts <= 1].index.tolist()
print("\n갯수가 1개 이하인 키워드들:")
low_frequency_keywords


갯수가 1개 이하인 키워드들:


[' 자동차제작증',
 '용인앱택시',
 ' 주상복합개발',
 ' 헝겊인형',
 '추정가',
 ' 정신의학적',
 ' 위촉후',
 ' 동양사',
 ' 최고지도자',
 ' 코인총판',
 ' 극초음속',
 ' 애버라인',
 ' 북수원점',
 ' 까치랠리',
 ' 용인시의회한편',
 ' 청명초등학교',
 ' 동백이닷컴',
 ' 진로활동',
 ' 사고확인',
 ' 경기도간',
 ' 사제단대표',
 ' 수도자대표',
 ' 뉴스속인물',
 ' 조연우',
 ' 투입코',
 ' 용인특례시청정부',
 '지산',
 ' 냉면사리',
 ' 인류문명',
 ' 냉기류',
 ' 나리백시티개발',
 ' 장비산업법',
 ' 유틸리티나',
 ' 남성티셔츠',
 ' 중순위',
 ' 분당수서간고속화도',
 ' 가사관리자들',
 ' 헬러윈',
 ' 골드만',
 ' 써모',
 ' 두산공작기계',
 ' 청주오창테크노폴리스',
 ' 월랑경로당',
 ' 원로사목사제',
 ' 유원지들',
 ' 당일형',
 '삼가',
 ' 장의자',
 ' 위생범',
 ' 현공화',
 ' 통제보호구역',
 ' 병자성사',
 ' 겨울훈련',
 ' 제초시기',
 ' 손밥상',
 ' 뉴스공장',
 ' 소신학교',
 ' 엠에스산업개발',
 ' 겸직내용',
 ' 무게별',
 ' 해임절차',
 ' 남호초',
 ' 신자분들',
 ' 실업자들',
 ' 레디앙',
 ' 목탁',
 ' 초하루',
 ' 도한영',
 ' 떨떠름',
 ' 공기여과기',
 ' 공조설비',
 ' 제일저축은행',
 ' 중재노력',
 ' 상생효과',
 ' 한국방정환재단',
 ' 이이재',
 ' 기업모집',
 '김현숙',
 ' 구매대금',
 ' 국립대구',
 ' 필요기술',
 ' 주창자',
 ' 보호접지',
 ' 전원공급장치',
 ' 나랏빚',
 ' 조영주',
 ' 외국인투자신고서',
 ' 내부회의',
 '야근',
 ' 한국프로농구연맹',
 ' 젖먹이',
 ' 정기주총',
 ' 단위부담금',
 ' 직원사망',
 ' 신차개발',
 ' 특수병실',
 ' 음압병실은기압',
 ' 

In [45]:
# '증오'가 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'\b증오\b', na=False)]

# 결과 출력
filtered_df.iloc[3]['키워드']

'이재명, 큰절, 눈물, 상처, 이재명, 더불어민주당, 대선, 후보, 연일, 반성, 사죄, 자신, 약점, 도덕성, 민주당, 사과, 중도층, 지지, 대선, 최대, 분기점, 연휴, 후보, 상태, 지지율, 정체, 타개, 위기감, 작용, 후보, 경기, 성남, 상대원, 시장, 자신, 유년기, 시절, 언급, 눈물, 자리, 이낙연, 민주당, 대표, 동행, 후보, 초등학교, 산꼭대기, 어머니, 화장실, 출근, 공장, 행복, 눈시울, 후보, 아버지, 시장, 청소, 노동자, 어머니, 시장, 공중, 화장실, 이용자, 소변, 대변, 설명, 화장실, 아들, 어머니, 거짓말, 판검사, 실력, 변호사, 노력, 자리, 상처, 토로, 후보, 형수, 욕설, 가족, 논란, 고개, 민주당, 반성, 강조, 후보, 경기, 용인시, 경기도, 공약, 윤호중, 원내, 대표, 민주당, 의원, 예정, 큰절, 후보, 사죄, 큰절, 후보, 민주당, 개혁, 진보, 세력, 핵심, 가치, 공정, 측면, 부족, 국민들, 남불, 질책, 생각, 관계자, 민주당, 선대위, 선대위, 현상, 지지율, 정체, 위기의식, 후보, 차원, 설명, 후보, 국민, 주장, 세대포위론, 적극, 반박, 세대포용, 세대포위론, 전폭적, 지지, 부모, 세대, 지지, 전략, 후보, 경기, 유세, 국민, 겨냥, 염장, 이익, 타인, 고통, 강요, 분열, 증오, 이용, 포위, 세대포위론, 비판, 박재현'

In [46]:
# '키워드' 열의 모든 단어들을 리스트로 모으기
all_keywords = []
train_df['키워드'].apply(lambda x: all_keywords.extend(x.split(',')))

# 모든 단어들의 빈도 계산
keyword_counts = pd.Series(all_keywords).value_counts()

# 갯수가 2개 이하인 키워드들 필터링
low_frequency_keywords = keyword_counts[keyword_counts == 2].index.tolist()
print("\n갯수가 2개인 키워드들:")
low_frequency_keywords


갯수가 2개인 키워드들:


[' 사업운전자금',
 ' 코탑',
 ' 와이더',
 ' 산타모자',
 ' 묘포장',
 ' 합산액',
 ' 렉스',
 ' 레이크머티리얼즈',
 ' 류정미',
 ' 신창재',
 ' 동래학춤',
 ' 영세업자들',
 ' 감지시스템',
 ' 오서',
 ' 환경영향평가협의회',
 ' 휴계시설',
 ' 포은아트홀서',
 ' 농약허용물질목록관리제도',
 ' 원재씨',
 ' 슈가슈',
 ' 표고차',
 ' 금오테크노밸리',
 ' 푸른컵',
 ' 뇌물혐의',
 '을지연습',
 ' 대북송금수사',
 ' 김동숙',
 ' 예약필수',
 '공원녹지기본계획',
 ' 보급ㆍ지원',
 ' 우왕전',
 ' 가족샤워장',
 ' 표절률',
 ' 광주시립미술관',
 ' 호안정비',
 ' 정상록',
 ' 패혈증',
 ' 추진반',
 ' 블루래빗',
 ' 학습기회',
 ' 정책활동',
 ' 이준오',
 ' 심의절차',
 ' 평원',
 ' 올슉업',
 ' 한국농업아카데미',
 ' 생활지음',
 ' 이호건',
 ' 음경택',
 ' 인권보호관',
 ' 접근시',
 ' 노대성',
 ' 국제장애인상',
 ' 세럼',
 ' 적령기',
 ' 빛고을로또',
 ' 활용률',
 ' 이법종교',
 ' 티이엠씨',
 ' 가족친화제도',
 ' 증평군의회',
 ' 물랑로티',
 ' 스타필드수원',
 ' 이원철',
 ' 훈국씨',
 ' 이지운',
 ' 평촌역',
 ' 구약성서',
 ' 근현대미술관',
 ' 더뮤즈',
 ' 구본교',
 '로비',
 ' 수온저감시설',
 ' 전당포',
 ' 문갑도',
 ' 부대장',
 ' 연결터널',
 ' 재도입',
 ' 군장병들',
 ' 경기용인서부경찰서',
 ' 중앙호수공원',
 '경기농기원',
 ' 택배노조원들',
 ' 지종철',
 ' 우두머리',
 ' 거출금',
 ' 적과',
 ' 해양위',
 ' 환노',
 ' 용인체력',
 ' 일왕저수지',
 ' 상하도서관',
 ' 낙동강체육공원',
 ' 석호철',
 ' 기안동',
 ' 만석거',
 '물건',
 ' 수진씨',
 ' 주민숙원사업',
 '

In [32]:
import pandas as pd

# '키워드' 열의 모든 단어들을 리스트로 모으기
all_keywords = []
train_df['키워드'].apply(lambda x: all_keywords.extend(x.split(',')))

# 모든 단어들의 빈도 계산
keyword_counts = pd.Series(all_keywords).value_counts()

# 갯수가 3개 이하인 키워드들 필터링
low_frequency_keywords = keyword_counts[keyword_counts <= 3].index.tolist()
print("\n갯수가 3개 이하인 키워드들:")
print(low_frequency_keywords)

# 갯수가 3개 이하인 키워드들을 제거하는 함수
def remove_low_frequency_keywords(keywords):
    valid_keywords = [word.strip() for word in keywords.split(',') if word.strip() not in low_frequency_keywords]
    return ', '.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드'] = train_df['키워드'].apply(remove_low_frequency_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_low_frequency_keywords)

# 결과 출력
print("\n수정된 데이터프레임:")
print(train_df)


갯수가 3개 이하인 키워드들:
[' 동그랑땡', ' 견학비', ' 문화체험활동', ' 한국생활', ' 김명돌', ' 남자간호사', ' 약취', ' 플러그인하이브리드', ' 인천대교', ' 여의도고', ' 강북횡단선', ' 고유생태계', ' 남자프로농구', ' 카핑', ' 의왕간', ' 김윤교', ' 사회사업팀장', ' 숭상', ' 결혼지원혁명', ' 업무구역', ' 황곡초등학교', ' 행정안전부장관상', ' 대전고', ' 반월역', ' 시민사회수석실', ' 빅뱅이', ' 천안성성비스타', ' 골프꿈나무들', ' 용인에버랜드역칸타빌', ' 바닥슬래브', ' 수지한성', ' 성과포럼', ' 최지성', ' 프루지오', ' 자녀가구', ' 업무공백', ' 활동증명서', ' 증산동', ' 지역공동체과장', ' 국제협력관', ' 나현빈', ' 데시앙네스트', ' 금속가공', ' 전주에코시티', ' 제설자재', ' 까망베르', ' 김윤혁', ' 존배', ' 일반국도', ' 제직', ' 캐스트', '형형색색', '분야', ' 군법', ' 월경곤란증', ' 젠더폭력', ' 기회수도파트너스', ' 쌍욕일탈', ' 특정성', ' 원로목사', ' 서울집값', ' 지역문화시설', ' 원삼간', ' 인주초등학교', ' 보안시설', ' 평생학습마을연합', ' 강원도민들', ' 집단반발', '동두천시', ' 박정오', ' 전국각지', ' 월드컵공원', ' 청명센트레빌아파트', ' 단일군락', ' 단속장비', ' 포설', ' 식품가공업체', ' 법원판결', ' 하남드림', ' 사회상규', ' 선행판결', ' 대우트럼프월드', ' 한국협상학회', ' 소통마당', ' 변성칠', ' 장기사용', ' 이국주', ' 도람근린공원', ' 향취', ' 김승준', ' 광주북구청', ' 립밤', ' 문해시화', ' 문해학습자들', ' 동두천시체육', ' 아름마을', ' 일일찻집', ' 이탈자', ' 박동성', ' 육서현', ' 권선구보건소', ' 둥지탑', ' 복지이슈포커스', ' 한국크리스토퍼', ' 

KeyboardInterrupt: 